In [1]:
import sys
from pathlib import Path

In [2]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

2022-11-02 21:56:02.785291: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-02 21:56:02.948663: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-02 21:56:03.493572: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2022-11-02 21:56:03.493662: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: can

In [3]:
sys.path.append(str(Path("../../ssl").resolve()))

In [4]:
from src.models.pi_model.pi_model import PiModel
from src.models.pi_model.pi_model_config import PiModelConfig
from src.trainers.basic.categorical_ce import CategoricalCETrainer
from src.trainers.basic.categorical_ce_config import CategoricalCETrainerConfig
from src.data_loaders.basic.categorical_ce_config import CategoricalCEDataLoaderConfig
from src.data_loaders.basic.categorical_ce import CategoricalCEDataLoader

# Description

In this notebook, the AlexNet model will be trained on the CIFAR-10 dataset using only 25% of the labelled dataset.

## Set up Experiment

In [5]:
class TrainerConfig(CategoricalCETrainerConfig):
    num_epochs = 150

train_config = TrainerConfig()

In [6]:
class ModelConfig(PiModelConfig):
    input_shape = (32, 32, 3)
    output_shape = 10

model_config = ModelConfig()

In [7]:
class DataLoaderConfig(CategoricalCEDataLoaderConfig):
    batch_size = 64
    num_classes = 10
    
    # same augmenters as pi model, for a fair comparison
    blur_params = {
        'chance': 0.10,
        'kernel_ratio': 0.10,
        'blur_strength': (0.1, 2.0)
    }

    crop_params = {
        'chance': 0.50,
        'crop_size': (0.08, 1.0),
        'aspect_range': (0.75, 1.33),
        'num_tries': 100
    }

    jitter_params = {
        'chance': 0.50,
        'distort_strength': 0.20,
        'drop_chance': 0.05
    }

data_loader_config = DataLoaderConfig()

## Get Datasets

In [8]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.cifar10.load_data()

In [9]:
# get train dataset by subsampling 10% of the full training dataset (stratified by labels)
_, x_train, _, y_train = train_test_split(
    x_train_full, y_train_full,
    stratify = y_train_full,
    test_size = 0.20, random_state = 42
)

In [10]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_data = tf.data.Dataset.from_tensor_slices((x_test_full, y_test_full))

2022-11-02 21:56:05.383657: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 21:56:05.390650: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 21:56:05.391135: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 21:56:05.392035: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [11]:
# create train dataset
train_data = CategoricalCEDataLoader(train_data, data_loader_config)(training = True)

In [12]:
# create test dataset
val_data = CategoricalCEDataLoader(val_data, data_loader_config)(training = False)

# Train Model

In [13]:
model = PiModel(model_config)()

In [14]:
trainer = CategoricalCETrainer(
    model, train_data, train_config,
    val_dataset = val_data)

In [15]:
trainer.train()

2022-11-02 21:56:07.424918: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


Training loss at epoch 0 is : 2.19. Validation loss is : 2.00. Validation acc. is : 23.73.
Training loss at epoch 1 is : 1.98. Validation loss is : 1.81. Validation acc. is : 29.56.
Training loss at epoch 2 is : 1.88. Validation loss is : 1.71. Validation acc. is : 35.24.
Training loss at epoch 3 is : 1.80. Validation loss is : 1.70. Validation acc. is : 36.75.
Training loss at epoch 4 is : 1.76. Validation loss is : 1.62. Validation acc. is : 40.60.
Training loss at epoch 5 is : 1.75. Validation loss is : 1.59. Validation acc. is : 41.02.
Training loss at epoch 6 is : 1.72. Validation loss is : 1.61. Validation acc. is : 40.66.
Training loss at epoch 7 is : 1.69. Validation loss is : 1.54. Validation acc. is : 43.08.
Training loss at epoch 8 is : 1.65. Validation loss is : 1.50. Validation acc. is : 43.89.
Training loss at epoch 9 is : 1.62. Validation loss is : 1.46. Validation acc. is : 46.44.
Training loss at epoch 10 is : 1.62. Validation loss is : 1.44. Validation acc. is : 47.69